In [1]:
# %%
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os


# %%
def generate_gan_dataset(generator, latent_dim, label_vector, num_samples=1000, save_path="gan_generated_dataset.npz"):
    """
    Generate synthetic light curves from a trained conditional GAN and save to .npz file.

    Parameters:
    - generator: Trained GAN generator model
    - latent_dim: Dimension of the noise vector
    - label_vector: List [fc, amp, is_qpo]
    - num_samples: Number of samples to generate
    - save_path: Where to save the output dataset
    """
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # 1. Prepare latent vectors and labels
    noise = tf.random.normal([num_samples, latent_dim])
    labels = tf.convert_to_tensor([label_vector] * num_samples, dtype=tf.float32)

    # 2. Generate sequences
    print(f"🌀 Generating {num_samples} synthetic sequences for label: {label_vector}")
    generated_series = generator([noise, labels], training=False).numpy()  # shape: (N, 512, 1)

    # 3. Save to .npz
    print(f"💾 Saving dataset to {save_path}...")
    np.savez_compressed(save_path, X=generated_series, y=np.array([label_vector] * num_samples))
    print("✅ Done!")
    return generated_series


# %%
# Load trained generator
generator = load_model("saved_models/qpo_cgan_generator")

# Example usage: generate QPO samples
qpo_label = [0.35, 0.8, 1.0]  # fc=0.35 Hz, amp=0.8, is_qpo=1
generate_gan_dataset(generator, latent_dim=100, label_vector=qpo_label,
                     num_samples=2000, save_path="gan_generated/qpo_fc035_amp08.npz")

# Example usage: generate noise-only (non-QPO) samples
non_qpo_label = [0.35, 0.8, 0.0]  # Same fc/amp but no QPO
generate_gan_dataset(generator, latent_dim=100, label_vector=non_qpo_label,
                     num_samples=2000, save_path="gan_generated/noise_fc035_amp08.npz")

# %%
# Optional: sweep over multiple [fc, amp] combinations
fc_vals = [0.2, 0.35, 0.5]
amp_vals = [0.3, 0.6, 0.9]

for fc in fc_vals:
    for amp in amp_vals:
        for is_qpo in [0.0, 1.0]:
            label = [fc, amp, is_qpo]
            tag = f"fc{fc:.2f}_amp{amp:.2f}_qpo{int(is_qpo)}"
            save_path = f"gan_generated/sweep/{tag}.npz"
            generate_gan_dataset(generator, 100, label, 500, save_path)


ValueError: File format not supported: filepath=saved_models/qpo_cgan_generator. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(saved_models/qpo_cgan_generator, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).